In [131]:
!pip install medmnist
!pip install flwr[simulation]
!pip install tenseal

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [132]:
import numpy as np
from collections import OrderedDict
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import transforms
from medmnist import PathMNIST
import flwr as fl
from typing import Dict, List, Tuple
from flwr.common import Context, Metrics, ndarrays_to_parameters
from flwr.client import Client, ClientApp, NumPyClient
from flwr.common import log
from flwr.common import Scalar,NDArrays
from flwr.server import ServerApp, ServerConfig, ServerAppComponents
from flwr.server.strategy import FedAvg
from flwr.simulation import run_simulation
from torchvision.models import resnet18,ResNet18_Weights,efficientnet_b0, EfficientNet_B0_Weights,mobilenet_v3_small,MobileNet_V3_Small_Weights
import tenseal as ts
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Flower {fl.__version__}")

Flower 1.15.2


# `1.Define THE  MODEL`

In [133]:

num_classes = 9

modeleff = efficientnet_b0(weights=EfficientNet_B0_Weights.IMAGENET1K_V1)
modeleff.classifier[1] = nn.Linear(modeleff.classifier[1].in_features, num_classes)
modelefficient=modeleff.to(device)


In [134]:
modelnetV3 = mobilenet_v3_small(weights=MobileNet_V3_Small_Weights.DEFAULT)
modelnetV3.classifier[3] = nn.Linear(modelnetV3.classifier[3].in_features, num_classes)
modelnetV3 = modelnetV3.to(device)

# `2.Traning & validation Function`

In [135]:

def train_epoch(model, train_loader):
    """Train model for one epoch."""
    model.train()
    correct, total, running_loss = 0,0,0.0
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.squeeze().to(device)
        optimizer.zero_grad() # Clear gradients
        outputs = model(inputs)# Forward pass
        loss = criterion(outputs, labels)# Compute loss
        loss.backward() # Backward pass (compute gradients)
        optimizer.step()# Update weights

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_acc = 100 * correct / total
    return train_loss, train_acc


def validate(model, val_loader):
    """Validate model on validation set."""
    model.eval() # Set the model to evaluation mode
    correct, total, val_loss = 0,0,0.0
    criterion = nn.CrossEntropyLoss()
    with torch.no_grad():  # Disable gradient computation
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.squeeze().to(device)
            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, labels)# Compute loss

            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1) # Get predicted class
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    val_loss = val_loss / len(val_loader)
    val_acc = 100 * correct / total
    return val_loss, val_acc


## `Train and Validate  the  model `

In [136]:
def train_model(model, train_loader, val_loader, num_epochs):
    """Full training loop with validation."""
    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []

    for epoch in range(num_epochs):
        train_loss, train_acc = train_epoch(model, train_loader)
        train_losses.append(train_loss)
        train_accuracies.append(train_acc)
        val_loss, val_acc = validate(model, val_loader)
        # Store metrics
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)

        print(f"Epoch {epoch+1}/{num_epochs}: "
              f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% | "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

    return train_losses, train_accuracies, val_losses, val_accuracies

# `3.Evaluation THE Model`

In [137]:

def evaluate_model(model, test_loader):
    """Evaluate model on test set and return predictions/labels."""
    model.eval()
    total = 0
    correct = 0
    test_loss = 0.0
    all_preds = []
    all_labels = []
    criterion = nn.CrossEntropyLoss()
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device),labels.squeeze().to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            test_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            total += labels.size(0)
            correct += preds.eq(labels).sum().item()

    test_loss = test_loss / len(test_loader)
    test_acc = 100 * correct / total
    return test_loss,test_acc,np.array(all_preds), np.array(all_labels)

# `4.Lead THE DATASET`

In [138]:

# Set random seeds for reproducibility
torch.manual_seed(42)
NUM_CLIENTS = 10
BATCH_SIZE = 64


In [139]:
def load_full_datasets():
    """
    Load the complete PathMNIST dataset splits with appropriate transformations.

    Returns:
        tuple: (train_dataset, val_dataset, test_dataset)
    """
    # Transformations for training data (with augmentation)
    train_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ColorJitter(0.2, 0.2, 0.2, 0.1),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

    # Transformations for validation and testing (without augmentation)
    test_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

    # Load the full datasets
    train_dataset = PathMNIST(split='train', transform=train_transform, download=True)
    val_dataset = PathMNIST(split='val', transform=test_transform, download=True)
    test_dataset = PathMNIST(split='test', transform=test_transform, download=True)

    return train_dataset, val_dataset, test_dataset

In [140]:
def partition_dataset(dataset, num_partitions: int):
    """
    Partition a dataset into a specified number of subsets.

    Args:
        dataset: The complete dataset to partition.
        num_partitions (int): Number of partitions (e.g., number of clients).

    Returns:
        list: A list of Subset objects, each corresponding to one partition.
    """
    # Shuffle dataset indices randomly
    indices = np.random.permutation(len(dataset))
    # Split indices into num_partitions parts
    partitions_indices = np.array_split(indices, num_partitions)
    # Create a Subset for each partition
    subsets = [Subset(dataset, idx) for idx in partitions_indices]
    return subsets

In [141]:
def partition_dataset_dirichlet(dataset, num_partitions: int, alpha: float = 0.5):
    """
    Partition a dataset using a Dirichlet distribution for a non-IID split.

    Args:
        dataset: The complete dataset to partition.
        num_partitions (int): Number of partitions (e.g., number of clients).
        alpha (float): Concentration parameter for the Dirichlet distribution.
                       Smaller values yield a more non-IID split.

    Returns:
        list: A list of Subset objects, each corresponding to one partition.
    """
    # Extract labels from the dataset
    targets = [
        int(dataset[i][1].item()) if hasattr(dataset[i][1], 'item') else int(dataset[i][1])
        for i in range(len(dataset))
    ]
    targets = np.array(targets)
    classes = np.unique(targets)

    # Initialize lists to store indices for each client
    client_indices = [[] for _ in range(num_partitions)]

    # For each class, split the indices among clients based on Dirichlet proportions
    for c in classes:
        idx_c = np.where(targets == c)[0]
        np.random.shuffle(idx_c)

        # Sample a probability vector from the Dirichlet distribution
        proportions = np.random.dirichlet(np.repeat(alpha, num_partitions))

        # Determine the number of samples for each client from this class
        # First, get the raw counts (rounded down)
        counts = (proportions * len(idx_c)).astype(int)
        # Adjust counts so that they sum to the total number of samples in this class
        diff = len(idx_c) - np.sum(counts)
        for d in range(diff):
            counts[d % num_partitions] += 1

        # Split indices according to these counts
        splits = np.split(idx_c, np.cumsum(counts)[:-1])
        for i, split in enumerate(splits):
            client_indices[i].extend(split.tolist())

    # Create a Subset for each client
    subsets = [Subset(dataset, indices) for indices in client_indices]
    return subsets


In [142]:
def load_client_datasets(partition_id: int, num_clients: int = NUM_CLIENTS):
    """
    Load the client-specific DataLoaders by partitioning the training and validation datasets.

    Args:
        partition_id (int): The client index (from 0 to num_clients-1).
        num_clients (int): Total number of clients.

    Returns:
        tuple: (train_loader, val_loader, test_loader)
    """
    # Load the full datasets
    train_dataset, val_dataset, test_dataset = load_full_datasets()

    # Partition the training and validation datasets
    train_subsets = partition_dataset(train_dataset, num_clients)
    val_subsets = partition_dataset(val_dataset, num_clients)

    # Retrieve the subset for the specific client
    client_train = train_subsets[partition_id]
    client_val = val_subsets[partition_id]

    # Create DataLoaders for each dataset
    train_loader = DataLoader(client_train, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
    val_loader = DataLoader(client_val, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

    return train_loader, val_loader, test_loader

In [143]:
import matplotlib.pyplot as plt
import seaborn as sns

def visualize_client_data_distribution(num_clients: int = NUM_CLIENTS,alpha: float = 0.5):
    """
    Visualize the distribution of dataset categories for each client.

    Loads the full training dataset, partitions it among the specified number of clients,
    and then creates a bar plot for each client showing the count of samples for each class.
    """
    # Load the full training dataset (using the same transforms as before)
    train_dataset, _, _ = load_full_datasets()

    # Partition the training dataset among the clients
    train_subsets = partition_dataset(train_dataset, num_clients)
    #train_subsets=partition_dataset_dirichlet(train_dataset, num_clients, alpha)

    # Set up the plot grid: e.g., 2 rows x 5 columns for 10 clients
    fig, axs = plt.subplots(nrows=2, ncols=5, figsize=(20, 8))
    axs = axs.flatten()

    # Iterate through each client's subset
    for i, subset in enumerate(train_subsets):
        indices = subset.indices

        # Retrieve labels from the original train_dataset using these indices.
        # Use .item() if available to ensure a scalar value.
        labels = []
        for idx in indices:
            label_val = train_dataset[idx][1]
            if hasattr(label_val, 'item'):
                label_val = label_val.item()
            labels.append(int(label_val))

        # Count occurrences for each class (from 0 to num_classes-1)
        counts = [labels.count(cls) for cls in range(num_classes)]

        # Create a bar plot for the client.
        # We now pass the x variable to 'hue' and set dodge=False and legend=False to avoid warnings.
        sns.barplot(
            x=list(range(num_classes)),
            y=counts,
            hue=list(range(num_classes)),
            dodge=False,
            ax=axs[i],
            palette="viridis",
            legend=False
        )
        axs[i].set_title(f'Client {i} Distribution')
        axs[i].set_xlabel('Class')
        axs[i].set_ylabel('Count')

    plt.tight_layout()
    plt.show()

# Call the visualization function to display the plots
'''visualize_client_data_distribution()'''


'visualize_client_data_distribution()'

# 6.Federated Learning With FLOWER

In [144]:
def get_parameters(model) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in model.state_dict().items()]

def set_parameters(model, parameters: List[np.ndarray]):
    params_dict = zip(model.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
    model.load_state_dict(state_dict)


In [145]:
# Context for CKKS encryption
context = ts.context(
    ts.SCHEME_TYPE.CKKS,
    poly_modulus_degree=32768,  # Degree of the polynomial modulus
    coeff_mod_bit_sizes=[60, 40, 40, 60]  # Coefficient modulus sizes
)
context.generate_galois_keys()  # Generate Galois keys for rotation operations
context.global_scale = 2**40  # Set global scale for precision

def encrypt_weights(weights: List[np.ndarray]) -> List[ts.CKKSVector]:
      """
      Encrypt a list of NumPy arrays (model weights) using CKKS.
      """
      return [ts.ckks_vector(context, weight.flatten()) for weight in weights]

def decrypt_weights(encrypted_weights: List[ts.CKKSVector], original_weights: List[np.ndarray]) -> List[np.ndarray]:
    """
    Decrypt a list of CKKS vectors back into NumPy arrays.
    Args:
        encrypted_weights: List of encrypted weights (CKKS vectors).
        original_weights: List of original weights (NumPy arrays) to get the shapes.
    Returns:
        List of decrypted weights as NumPy arrays.
    """
    return [
        np.array(enc_weight.decrypt()).reshape(original_weight.shape)  # Convert to NumPy array
        for enc_weight, original_weight in zip(encrypted_weights, original_weights)
    ]

In [ ]:
# Extract initial weights from the model
initial_weights = get_parameters(modeleff)
print(initial_weights)

In [ ]:
# Encrypt the weights
encrypted_weights = encrypt_weights(initial_weights)
print("Weights encrypted successfully!")

In [ ]:
# Decrypt the weights
decrypted_weights = decrypt_weights(encrypted_weights, initial_weights)
print("Weights decrypted successfully!")
print(decrypted_weights)

# Verify integrity
for orig, decrypted in zip(initial_weights, decrypted_weights):
    assert np.allclose(orig, decrypted, atol=1e-3), "Decrypted weights do not match original weights!"

print("Encryption and decryption verified successfully!")

### 6.1. Define the Flower ClientApp

In [ ]:
class FlowerClient(NumPyClient):
    def __init__(self,partition_id,model, train_loader, val_loader):
        self.partition_id = partition_id
        self.model = model
        self.train_loader = train_loader
        self.val_loader = val_loader


    def get_parameters(self, config) :
        #get the model parameters and return them as a list of NumPy ndarray’s
        print(f"[Client {self.partition_id}] get_parameters")
        return get_parameters(self.model)

    def fit(self, parameters: List[np.ndarray], config: Dict) -> Tuple[List[np.ndarray], int, Dict]:
        print(f"[Client {self.partition_id}] fit, config: {config}")
        set_parameters(self.model, parameters)#Update the parameters of the local model with the parameters received from the server
        train_loss, train_acc = train_epoch(self.model, self.train_loader)#Train the updated model on the local train dataset
        return self.get_parameters({}), len(self.train_loader), {"train_loss": train_loss}

    def evaluate(self, parameters: List[np.ndarray], config: Dict) -> Tuple[float, int, Dict]:
        print(f"[Client {self.partition_id}] evaluate, config: {config}")
        set_parameters(self.model, parameters)
        val_loss, val_acc = validate(self.model, self.val_loader)
        return float(val_loss), len(self.val_loader), {"val_acc": float(val_acc)}

In [ ]:
def client_fn(context: Context) ->Client:
    """Create Flower client with partition ID."""
    model=modelnetV3
    partition_id = context.node_config["partition-id"]
    train_loader, val_loader, _ =load_client_datasets(partition_id)
    return FlowerClient(partition_id,model,train_loader, val_loader).to_client()

client_app = ClientApp(client_fn=client_fn)

### 6.2 function Aggregation validation accuracy across clients

In [ ]:
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    val_accs = [num_examples * m["val_acc"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]
    return {"val_acc": sum(val_accs) / sum(examples)}

In [ ]:
_, _, test_loader = load_client_datasets(0)
def evaluate_fn(
        server_round: int, parameters: NDArrays, config: Dict[str,Scalar],
    ) -> Tuple[float, Dict[str, Scalar]]:
        model = modelnetV3
        set_parameters(model, parameters)
        test_loss, test_acc, _, _ = evaluate_model(model, test_loader)
        print(f"Global Test Accuracy (Round {server_round}): {test_acc:.2f}%")
        return test_loss, {"test_acc": test_acc }


Using downloaded and verified file: /root/.medmnist/pathmnist.npz
Using downloaded and verified file: /root/.medmnist/pathmnist.npz
Using downloaded and verified file: /root/.medmnist/pathmnist.npz


In [ ]:
# Create an instance of the model and get the parameters
params = get_parameters(modelnetV3)

### 6.4. Define the Flower ServertApp

In [ ]:
def server_fn(context: Context) -> ServerAppComponents:
    # Create the FedAvg strategy
    strategy = FedAvg (
        fraction_fit= 1.0,
        min_available_clients=NUM_CLIENTS,
        evaluate_fn= evaluate_fn,
        initial_parameters=ndarrays_to_parameters(params),
        evaluate_metrics_aggregation_fn=weighted_average,
    )
    # Configure the server for n rounds of training
    config = ServerConfig(num_rounds=10)
    return ServerAppComponents(strategy=strategy, config=config)


# Create the ServerApp
server_app = ServerApp(server_fn=server_fn)

In [ ]:
# Specify the resources each of your clients need
# By default, each client will be allocated 1x CPU and 0x GPUs
backend_config = {"client_resources": {"num_cpus": 2, "num_gpus": 1.0}}

if device == "cuda":
    backend_config = {"client_resources": {"num_cpus": 1, "num_gpus": 1.0}}


### 6.5. Run Flower SIMULATION

In [ ]:
# Start simulation
run_simulation(
    server_app=server_app,
    client_app=client_app,
    num_supernodes=NUM_CLIENTS,
    backend_config=backend_config,
    )

INFO :      Starting Flower ServerApp, config: num_rounds=10, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters
(pid=26553) 2025-02-18 15:00:44.454719: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=26553) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(pid=26553) E0000 00:00:1739890844.491877   26553 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=26553) E0000 00:00:1739890844.503494   26553 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
INFO :      initial parameters (loss, other metrics): 2.241863235

Global Test Accuracy (Round 0): 5.53%
(ClientAppActor pid=26553) Using downloaded and verified file: /root/.medmnist/pathmnist.npz
(ClientAppActor pid=26553) Using downloaded and verified file: /root/.medmnist/pathmnist.npz
(ClientAppActor pid=26553) Using downloaded and verified file: /root/.medmnist/pathmnist.npz
(ClientAppActor pid=26553) [Client 0] fit, config: {}
(ClientAppActor pid=26553) [Client 0] get_parameters
(ClientAppActor pid=26553) Using downloaded and verified file: /root/.medmnist/pathmnist.npz
(ClientAppActor pid=26553) Using downloaded and verified file: /root/.medmnist/pathmnist.npz
(ClientAppActor pid=26553) Using downloaded and verified file: /root/.medmnist/pathmnist.npz
(ClientAppActor pid=26553) [Client 2] fit, config: {}
(ClientAppActor pid=26553) [Client 2] get_parameters
(ClientAppActor pid=26553) Using downloaded and verified file: /root/.medmnist/pathmnist.npz
(ClientAppActor pid=26553) Using downloaded and verified file: /root/.medmnist/pathmnist.npz
(Cli

INFO :      aggregate_fit: received 10 results and 0 failures


(ClientAppActor pid=26553) [Client 6] get_parameters


INFO :      fit progress: (1, 0.40018681377436205, {'test_acc': 86.54596100278552}, 214.85908473000018)
INFO :      configure_evaluate: strategy sampled 10 clients (out of 10)


...

INFO :      aggregate_fit: received 10 results and 0 failures


(ClientAppActor pid=26553) [Client 8] get_parameters


INFO :      fit progress: (10, 0.17149236405977106, {'test_acc': 94.33147632311977}, 2465.7005178500003)
INFO :      configure_evaluate: strategy sampled 10 clients (out of 10)


Global Test Accuracy (Round 10): 94.33%
(ClientAppActor pid=26553) Using downloaded and verified file: /root/.medmnist/pathmnist.npz
(ClientAppActor pid=26553) Using downloaded and verified file: /root/.medmnist/pathmnist.npz
(ClientAppActor pid=26553) Using downloaded and verified file: /root/.medmnist/pathmnist.npz
(ClientAppActor pid=26553) [Client 1] evaluate, config: {}
(ClientAppActor pid=26553) Using downloaded and verified file: /root/.medmnist/pathmnist.npz
(ClientAppActor pid=26553) Using downloaded and verified file: /root/.medmnist/pathmnist.npz
(ClientAppActor pid=26553) Using downloaded and verified file: /root/.medmnist/pathmnist.npz
(ClientAppActor pid=26553) [Client 2] evaluate, config: {}
(ClientAppActor pid=26553) Using downloaded and verified file: /root/.medmnist/pathmnist.npz
(ClientAppActor pid=26553) Using downloaded and verified file: /root/.medmnist/pathmnist.npz
(ClientAppActor pid=26553) Using downloaded and verified file: /root/.medmnist/pathmnist.npz
(Clie

INFO :      aggregate_evaluate: received 10 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 10 round(s) in 2506.77s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.35853739231824877
INFO :      		round 2: 0.20683862410951406
INFO :      		round 3: 0.13709339497145265
INFO :      		round 4: 0.10629255592357367
INFO :      		round 5: 0.1083076471753884
INFO :      		round 6: 0.08097355372738094
INFO :      		round 7: 0.09009558866964654
INFO :      		round 8: 0.07776632728928234
INFO :      		round 9: 0.07731109516462312
INFO :      		round 10: 0.07267823737929575
INFO :      	History (loss, centralized):
INFO :      		round 0: 2.2418632359631294
INFO :      		round 1: 0.40018681377436205
INFO :      		round 2: 0.2593838068763767
INFO :      		round 3: 0.18495029870387727
INFO :      		round 4: 0.16632407883890962
INFO :      		round 5: 0.1767552546431533
INFO :      		round 6: 0.15996927906454136
INFO :      		round 7: 0.173911468447